In [1]:
import warnings

import geopandas as gpd
import numpy as np
import pandas as pd
import requests
from arcgis2geojson import arcgis2geojson
from pkg_resources import packaging


In [2]:
GEOPANDAS_VERSION = packaging.version.parse(gpd.__version__)

In [3]:
def _get_json_safely(response):
    """
    Check for JSON response errors, and if all clear,
    return the JSON data
    """
    # bad status code
    if response.status_code != 200:
        response.raise_for_status()

    json = response.json()  # get the JSON
    if "error" in json:
        raise ValueError("Error: %s" % json["error"])

    return json

In [4]:
def get(url, fields=None, where=None, limit=None, **kwargs):
    """
    Scrape features from a ArcGIS Server REST API and return a
    geopandas GeoDataFrame.
    Parameters
    ----------
    url : str
        the REST API url for the Feature Service
    fields : list of str, optional
        the list of fields to include; the default behavior ('None')
        returns all fields
    where : str, optional
        a string specifying the selection clause to select a subset of
        data; the default behavior ('None') selects all data
    limit : int, optional
        limit the returned data to this many features
    Example
    -------
    >>> import esri2gpd
    >>> url = "https://services.arcgis.com/fLeGjb7u4uXqeF9q/arcgis/rest/services/Philadelphia_ZCTA_2018/FeatureServer/0"
    >>> gdf = esri2gpd.get(url, fields=['zip_code'], where="zip_code=19123")
    >>> gdf
    """
    # Get the max record count
    metadata = requests.get(url, params=dict(f="pjson")).json()
    max_record_count = metadata["maxRecordCount"]

    # default behavior matches all features
    if where is None:
        where = "1=1"
    if fields is None:
        fields = "*"
    else:
        fields = ", ".join(fields)

    # extract object IDs of features
    queryURL = f"{url}/query"

    # Get the total record count
    params = dict(where=where, returnCountOnly="true", f="json")
    response = requests.get(queryURL, params=params)
    total_size = _get_json_safely(response)["count"]

    # Check the limit
    if limit is not None:
        total_size = limit

    # params for this request
    resultOffset = 0
    params = dict(
        f="json",
        outSR="4326",
        outFields=fields,
        resultOffset=resultOffset,
        where=where,
        **kwargs,
    )

    calls = total_size // max_record_count
    if calls > 10:
        warnings.warn(
            f"Long download time — total download will require {calls} separate requests"
        )

    out = []
    while params["resultOffset"] < total_size:

        remaining = total_size - params["resultOffset"]
        if remaining < max_record_count:
            params["resultRecordCount"] = remaining

        # get raw features
        response = requests.get(queryURL, params=params)
        json = _get_json_safely(response)

        # convert to GeoJSON and save
        geojson = [arcgis2geojson(f) for f in json["features"]]
        if GEOPANDAS_VERSION >= packaging.version.parse("0.7"):
            gdf = gpd.GeoDataFrame.from_features(geojson, crs="EPSG:4326")
        else:
            gdf = gpd.GeoDataFrame.from_features(geojson, crs={"init": "epsg:4326"})
        out.append(gdf)

        params["resultOffset"] += len(out[-1])

    return pd.concat(out, axis=0).reset_index(drop=True)

In [17]:
url = "https://sampleserver6.arcgisonline.com/arcgis/rest/services/Energy/Geology/FeatureServer/1"

In [5]:
gdf = get(url)

,geometry,objectid,emp_id,fold_type,classification_confidence,strike,dip,dipdirection,plunge,trend,rake,measurement_confidence,method_measurement,globalid,comments,photo
0,POINT (-118.43530 34.45270),20644,65321,1,1,245.0,45.0,NaN,NaN,NaN,NaN,2,1,{0560DB5A-0AFF-4421-8992-F8E46CFF6761},None,None
1,POINT (-118.41444 34.44410),20645,65321,2,2,165.0,84.0,NaN,NaN,NaN,NaN,4,2,{FEA1E1B1-B032-4AFD-ADAE-AADCEEA30CA6},None,None
2,POINT (-118.39896 34.50297),20646,65321,2,4,360.0,24.0,NaN,NaN,NaN,NaN,1,4,{3A8E2190-76ED-4D08-B1E4-1A354C33F4F8},None,None
3,POINT (-118.01166 34.58441),20647,Tara,7,4,180.0,45.0,0.0,0.0,180.0,45.0,3,None,{975C5554-5AF6-4758-AD22-BDF7A45EC19F},This could be a mistake fold,None
4,POINT (-118.42498 34.44223),20648,123,1,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,{6CF954C6-5DBA-4FAC-910B-BD728CD2D801},None,None
5,POINT (-118.73210 34.33272),20649,None,12,None,NaN,NaN,NaN,NaN,NaN,NaN,None,None,{43EF1E29-52F1-40B1-8415-4EB76C70AD79},None,None


In [18]:
metadata = requests.get(url, params=dict(f="pjson")).json()
metadata

{'currentVersion': 10.71,
 'id': 1,
 'name': 'Fold (Point)',
 'type': 'Feature Layer',
 'parentLayer': None,
 'defaultVisibility': True,
 'minScale': 0,
 'maxScale': 0,
 'geometryType': 'esriGeometryPoint',
 'description': '',
 'copyrightText': '',
 'editFieldsInfo': None,
 'ownershipBasedAccessControlForFeatures': None,
 'syncCanReturnChanges': False,
 'relationships': [],
 'isDataVersioned': False,
 'isDataArchived': False,
 'isCoGoEnabled': False,
 'supportsRollbackOnFailureParameter': False,
 'archivingInfo': {'supportsQueryWithHistoricMoment': False,
  'startArchivingMoment': -1},
 'supportsStatistics': True,
 'supportsAdvancedQueries': True,
 'supportsValidateSQL': True,
 'supportsCoordinatesQuantization': True,
 'supportsCalculate': True,
 'advancedQueryCapabilities': {'supportsPagination': True,
  'supportsTrueCurve': True,
  'supportsQueryWithDistance': True,
  'supportsReturningQueryExtent': True,
  'supportsStatistics': True,
  'supportsHavingClause': True,
  'supportsOrderB

In [21]:
metadata['editFieldsInfo']

In [24]:
url = "https://services6.arcgis.com/bKYAIlQgwHslVRaK/ArcGIS/rest/services/CasesByRegion_ViewLayer/FeatureServer/0"

In [25]:
metadata = requests.get(url, params=dict(f="pjson")).json()
metadata

{'currentVersion': 10.81,
 'id': 0,
 'name': 'CasesByRegion',
 'type': 'Feature Layer',
 'serviceItemId': '2d657302ce9342e39ba5d76cde644ee6',
 'cacheMaxAge': 300,
 'isView': True,
 'isUpdatableView': True,
 'sourceSchemaChangesAllowed': True,
 'displayField': 'Region_Name_AR',
 'description': '',
 'copyrightText': '',
 'defaultVisibility': True,
 'editingInfo': {'lastEditDate': 1655549364754},
 'multiScaleGeometryInfo': {'levels': [1, 3, 5, 7, 9, 11, 13, 15],
  'generalizationType': 'DP'},
 'relationships': [],
 'isDataVersioned': False,
 'hasContingentValuesDefinition': False,
 'supportsAppend': True,
 'supportsCalculate': True,
 'supportsASyncCalculate': True,
 'supportsTruncate': False,
 'supportsAttachmentsByUploadId': True,
 'supportsAttachmentsResizing': True,
 'supportsRollbackOnFailureParameter': True,
 'supportsStatistics': True,
 'supportsExceedsLimitStatistics': True,
 'supportsAdvancedQueries': True,
 'supportsValidateSql': True,
 'supportsCoordinatesQuantization': True,
 '

In [26]:
metadata['editingInfo']

{'lastEditDate': 1655549364754}